# Field-aware Factorization Machines

在 Factorization Machines 的基础之上，做了些许修改。在 Factorization Machines 中，每一个特征 $f$ 对应唯一的向量 $f_i$，特征交叉的时候就是直接与另一个特征对应的向量 $f_j$ 点乘后作为交叉特征的系数。但是在 Field-aware Factorization Machines 中，对这种特征交叉做了一些修改，把特征划分到不同的 Field 上（假如有 n 个特征，然后把它们划分到了 $f$ 个域上。